In [1]:
import numpy as np
import copy
from io import StringIO
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
dataset=StringIO("""Date,Open,High,Low,Close,Volume,Trade_count,Vwap
2015-12-01 09:00:00+00:00,118.88,118.94,118.88,118.94,1145,5,118.902052
2015-12-01 09:15:00+00:00,118.77,118.77,118.77,118.77,200,1,118.77
2015-12-01 09:30:00+00:00,118.69,118.69,118.6,118.6,900,4,118.61
2015-12-01 09:45:00+00:00,118.64,118.65,118.64,118.65,3580,5,118.648883
2015-12-01 10:00:00+00:00,118.65,118.65,118.55,118.55,1820,4,118.611538
2015-12-01 10:15:00+00:00,118.55,118.6,118.55,118.6,880,5,118.5625
2015-12-01 10:30:00+00:00,118.55,118.55,118.5,118.5,1878,5,118.513312
2015-12-01 10:45:00+00:00,118.59,118.72,118.59,118.72,2499,10,118.628431
2015-12-01 11:00:00+00:00,118.71,118.9,118.71,118.9,2842,11,118.86064
2015-12-01 11:15:00+00:00,118.87,118.87,118.87,118.87,300,2,118.87
2015-12-01 11:30:00+00:00,118.78,118.8,118.76,118.8,3914,22,118.785876
2015-12-01 11:45:00+00:00,118.8,118.99,118.77,118.9,7900,37,118.893542
2015-12-01 12:00:00+00:00,118.88,118.98,118.84,118.84,6540,34,118.922648
2015-12-01 12:15:00+00:00,118.82,118.84,118.77,118.77,5603,28,118.804962
2015-12-01 12:30:00+00:00,118.77,118.89,118.76,118.88,7612,31,118.824002
""")
df = pd.read_table(dataset, sep=",")

#ip = np.array([ [1,2,3],[6,8,9],[3,4,5],[4,7,8],[4,2,5],[5,7,4] ])
#op = np.array([[2,8,4,7,2,4]])
#op = op.reshape(6,1)
ip = np.array([ [1],  [2],  [0],  [2],  [0],  [1],  [2],  [1] ])
op = np.array([ [300],[100],[200],[100],[200],[300],[100],[300] ])
num_steps = 3
num_features = 3
#ip_shaped = np.reshape(ip, newshape=(-1, num_steps, num_features))

#X = np.array([ [1,2,3, 4, 5, 6] ])
#Y = np.array([[2,3,4,5,6,7]])
print(ip)
print("--")
#ip= np.tile(ip,(50,1))
#op = np.tile(op,(50,1))
print(len(op))
#ip = np.array([ [1],[1],[1],[1],[1],[2],[1],[2]])
#op = np.array([ [1,    0,    1,    0,    1,    1,    0,    1 ]]).T

#ip = np.array([ [1,1,2],[1,2,1],[1,0,2],[1,2,1],[1,0,2],[1,1,2],[1,2,1],[1,1,2]])
#op = np.array([ [1,    0,    1,    0,    1,    1,    0,    1 ]]).T
ip = np.array([ [1,1,1,1,1],[0,2,2,2,2],[1,3,3,3,3],[1,4,4,4,4],[0,5,5,5,5],[0,6,6,6,6],[1,7,7,7,7],[0,8,8,8,8]])
op = np.array([ [1,          0,          1,          1,          0,          0,          1,          0 ]]).T
_, numFeats = ip.shape

[[1]
 [2]
 [0]
 [2]
 [0]
 [1]
 [2]
 [1]]
--
8


In [42]:
def lstm_data_transform(x_data, y_data, timeSteps=2):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    #print(x_data.shape[0])
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + timeSteps

        # if index is larger than the size of the dataset, we stop
        #print(end_ix)
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        #print(x_data[i:end_ix])
        # Get only the last element of the sequency for y
        #print(y_data[end_ix])
        seq_y = y_data[i:end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

In [43]:
timeSteps = 3

ipt,opt=lstm_data_transform(ip,op, timeSteps)
print(ipt)
print(ipt.shape)
print(ipt[0].shape)
print(opt.shape)

print("====")
print(opt)
batchSize = len(ipt)


[[[1 1 1 1 1]
  [0 2 2 2 2]
  [1 3 3 3 3]]

 [[0 2 2 2 2]
  [1 3 3 3 3]
  [1 4 4 4 4]]

 [[1 3 3 3 3]
  [1 4 4 4 4]
  [0 5 5 5 5]]

 [[1 4 4 4 4]
  [0 5 5 5 5]
  [0 6 6 6 6]]

 [[0 5 5 5 5]
  [0 6 6 6 6]
  [1 7 7 7 7]]]
(5, 3, 5)
(3, 5)
(5, 3, 1)
====
[[[1]
  [0]
  [1]]

 [[0]
  [1]
  [1]]

 [[1]
  [1]
  [0]]

 [[1]
  [0]
  [0]]

 [[0]
  [0]
  [1]]]


In [5]:
# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output




In [77]:
import numpy as np
hd=50
ipd = timeSteps
od = 1

alpha = .1

print("\nTraining With Alpha:" + str(alpha))

# randomly initialize our weights with mean 0
wxh = 2*np.random.random((numFeats,hd)) - 1
who = 2*np.random.random((hd,od)) - 1
whh = 2*np.random.random((hd,hd)) - 1

wxhupd = np.zeros_like(wxh)
whoupd = np.zeros_like(who)
whhupd = np.zeros_like(whh)

def p(j, msg):
    if(j % 1000 == 0):
        display(msg)
        print("---")

for j in range(100001):
    count = 0
    l2_deltas = list()
    l1_values = list()
    l1_values.append(np.zeros(hd))
    
    d = np.zeros_like(opt)
    overallError = 0

    for inp in ipt:
        X = inp
        y = opt[count]
        count+=1
        prevHidWeights = l1_values[-1]
  
        inpWeights = np.dot(X,wxh)
        hidWeights = np.dot(prevHidWeights,whh)
        # hidden layer (input ~+ prev_hidden)
        l1 = sigmoid(inpWeights + hidWeights)
        # output layer (new price)
        opWeights  = np.dot(l1,who)       
        l2 = sigmoid(opWeights)
        # how much did we miss the target value?
        #l2_error = (np.square(y - l2)).mean(axis=0)
        l2_error = y - l2
        
        l2_deltas.append((l2_error)*sigmoid_output_to_derivative(l2))
        overallError += np.abs(np.sum(l2_error))
    
        # decode estimate so we can print it out
        d[numFeats - count - 1] = np.round(l2[0][-1])

        # store hidden layer so we can use it in the next timestep
        l1_values.append(copy.deepcopy(l1))
        
        
    future_l1_delta = np.zeros(hd)
    count = 0
    for inp in reversed(ipt):
        X= inp
        y = opt[-count-1]

        l1 = l1_values[-count-1]
        prev_l1 = l1_values[-count-2]
        
        # error at output layer
        l2_delta = l2_deltas[-count-1]
        # error at hidden layer
        l1_delta = (future_l1_delta.dot(whh.T) + l2_delta.dot(who.T)) * sigmoid_output_to_derivative(l1)
        
        # let's update all our weights so we can try again
        whoupd += np.atleast_2d(l1).T.dot(l2_delta)
        if l1_delta.shape != prev_l1.shape:
            prev_l1 = np.tile(prev_l1, (timeSteps, 1))

        whhupd += np.atleast_2d(prev_l1).T.dot(l1_delta)
        wxhupd += X.T.dot(l1_delta)
        
        future_l1_delta = l1_delta
        count+=1

    wxh += wxhupd * alpha
    who += whoupd * alpha
    whh += whhupd * alpha    

    # print out progress
    if(j % 10000 == 0):
        print(j)
        print("Error:" + str(overallError))
        print("Prediction:")
        display(d.T)
        print("Real:")
        display(opt.T)
#         print("------------")

    
    wxhupd *= 0
    whoupd *= 0
    whhupd *= 0


Training With Alpha:0.1
0
Error:6.180464144676781
Prediction:


array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]])

Real:


array([[[1, 0, 1, 1, 0],
        [0, 1, 1, 0, 0],
        [1, 1, 0, 0, 1]]])

10000
Error:0.011603897421341375
Prediction:


...

array([[[1, 0, 1, 1, 0],
        [0, 1, 1, 0, 0],
        [1, 1, 0, 0, 1]]])

100000
Error:0.003014811502792387
Prediction:


array([[[1, 1, 0, 1, 0],
        [1, 1, 0, 1, 0],
        [1, 1, 0, 1, 0]]])

Real:


array([[[1, 0, 1, 1, 0],
        [0, 1, 1, 0, 0],
        [1, 1, 0, 0, 1]]])